In [ ]:
#calculating term frequency
from collections import Counter

#term frequencies for a document
def calculate_tf(tokens):
    #count the frequency of each token in the document
    tf = Counter(tokens)
    #convert the counts to the format (term_index, frequency)
    return [(indexed_vocabulary[word], freq) for word, freq in tf.items() if word in indexed_vocabulary]

indexed_vocabulary = {word: index for index, word in enumerate(sorted_vocabulary)}
#adding a colun for frequencies
df['TERM_FREQUENCIES'] = df['TOKENIZED_SECTION_TEXT'].apply(calculate_tf)

print(df[['TOKENIZED_SECTION_TEXT', 'TERM_FREQUENCIES']])

                                   TOKENIZED_SECTION_TEXT  \
0       ['', 'Anarchism, '', ', is, a, political, phil...   
1       ['', 'Autism, '', ', is, a, neurodevelopmental...   
2       [Percentage, of, diffusely, reflected, sunligh...   
3       [Writing, cursive, forms, of, A, '', ', A, '',...   
4       ['', 'Alabama, '', ', (, ), is, a, state, in, ...   
...                                                   ...   
816930  [anglian, combination, football, league, engli...   
816931  [national, theatre, oslo, one, norways, larges...   
816932  [midnight, runner, novel, jack, higgins, publi...   
816933  [thats, call, music, may, refer, least, two, d...   
816934  [vergulde, draeck, gilt, dragon, metre, tonne,...   

                                         TERM_FREQUENCIES  
0       [(1168158, 3), (840112, 4), (1371987, 1), (136...  
1       [(1168158, 10), (840112, 7), (1303422, 1), (10...  
2       [(568848, 1), (1325372, 16), (1017460, 1), (14...  
3       [(818457, 1), (9956

In [ ]:
#calculating idf
from collections import defaultdict

total_term_frequency = defaultdict(int)
#iterating through each document and updating the count of each
for tokens in df['TOKENIZED_SECTION_TEXT']:
    for token in tokens:
        if token in indexed_vocabulary:
            total_term_frequency[token] += 1

idf_df = pd.DataFrame(list(total_term_frequency.items()), columns=['Term', 'Total_Term_Frequency'])

print(idf_df)


               Term  Total_Term_Frequency
0                is               1221085
1                 a               1687613
2         political                 34127
3        philosophy                  6526
4              that                314609
...             ...                   ...
1674119  hardnheavy                     1
1674120      cnsobu                     1
1674121   jorsalfar                     1
1674122    vergulde                     1
1674123      draeck                     1

[1674124 rows x 2 columns]


In [ ]:
#calculating weights
import math

#alculating Document Frequencies (DF) for IDF calculation
document_frequency = defaultdict(int)
for tokens in df['TOKENIZED_SECTION_TEXT']:
    unique_tokens = set(tokens)
    for token in unique_tokens:
        if token in indexed_vocabulary:
            document_frequency[token] += 1

#calculating IDF
N = len(df)  # Total number of documents
idf = {term: math.log(N / df_value) for term, df_value in document_frequency.items()}

#calculating TF/IDF weights based on your original TF calculation method
def calculate_tf_idf(term_frequencies):
    # term_frequencies is a list of tuples (term_index, frequency) from your TF calculation
    tf_idf_weights = [(term_index, freq * idf[indexed_vocabulary_inv[term_index]])
                      for term_index, freq in term_frequencies
                      if indexed_vocabulary_inv[term_index] in idf]
    return tf_idf_weights

#inverse mapping
indexed_vocabulary_inv = {v: k for k, v in indexed_vocabulary.items()}

#applying calculation
df['TF_IDF_WEIGHTS'] = df['TERM_FREQUENCIES'].apply(calculate_tf_idf)


print(df[['TOKENIZED_SECTION_TEXT', 'TF_IDF_WEIGHTS']])

                                   TOKENIZED_SECTION_TEXT  \
0       ['', 'Anarchism, '', ', is, a, political, phil...   
1       ['', 'Autism, '', ', is, a, neurodevelopmental...   
2       [Percentage, of, diffusely, reflected, sunligh...   
3       [Writing, cursive, forms, of, A, '', ', A, '',...   
4       ['', 'Alabama, '', ', (, ), is, a, state, in, ...   
...                                                   ...   
816930  [anglian, combination, football, league, engli...   
816931  [national, theatre, oslo, one, norways, larges...   
816932  [midnight, runner, novel, jack, higgins, publi...   
816933  [thats, call, music, may, refer, least, two, d...   
816934  [vergulde, draeck, gilt, dragon, metre, tonne,...   

                                           TF_IDF_WEIGHTS  
0       [(1168158, 1.7232534871538019), (840112, 1.772...  
1       [(1168158, 5.744178290512672), (840112, 3.1023...  
2       [(568848, 10.08695428669744), (1325372, 6.8397...  
3       [(818457, 7.1868263

In [ ]:
#uses cosine to calculate similarity

from collections import defaultdict

# # Initialize a dictionary to count the document frequency of each term
# document_frequency = defaultdict(int)

# # Calculate document frequency for each term
# for tokens in df['TOKENIZED_SECTION_TEXT']:
#     unique_tokens = set(tokens)
#     for token in unique_tokens:
#         if token in indexed_vocabulary:
#             document_frequency[token] += 1

# Calculate IDF values
N = len(df)  # Total number of documents
idf = {term: math.log(N / df_value) for term, df_value in document_frequency.items()}

# User Query and Relevance Calculation, using hardcoded query
user_query = "information retrieval model"
query_tokens = user_query.split()

# Calculate TF-IDF for query
tf_idf_query = [(indexed_vocabulary[word], query_tokens.count(word) * idf[word]) for word in query_tokens if word in indexed_vocabulary]

# Calculate relevance scores for each document
relevance_scores = []
for idx, row in df.iterrows():
    tf_idf_doc = row['TF_IDF_WEIGHTS']
    similarity_score = calculate_cosine_similarity(tf_idf_doc, tf_idf_query)
    relevance_scores.append((row['ARTICLE_ID'], similarity_score))

# Sort documents by relevance score
relevance_scores = sorted(relevance_scores, key=lambda x: x[1], reverse=True)

# Print top k relevant documents
top_k = 5
print(f"Top {top_k} Relevant Documents for Query: '{user_query}':")
for doc_id, score in relevance_scores[:top_k]:
    print(f"Document ID: {doc_id}, Relevance Score: {score}")


'from collections import defaultdict\nimport pandas as pd\nimport math\n\n# Initialize a dictionary to count the document frequency of each term\ndocument_frequency = defaultdict(int)\n\n# Calculate document frequency for each term\nfor tokens in df[\'TOKENIZED_SECTION_TEXT\']:\n    unique_tokens = set(tokens)\n    for token in unique_tokens:\n        if token in indexed_vocabulary:\n            document_frequency[token] += 1\n\n# Calculate IDF values\nN = len(df)  # Total number of documents\nidf = {term: math.log(N / df_value) for term, df_value in document_frequency.items()}\n\n# User Query and Relevance Calculation\nuser_query = "information retrieval model"\nquery_tokens = user_query.split()\n\n# Calculate TF-IDF for query\ntf_idf_query = [(indexed_vocabulary[word], query_tokens.count(word) * idf[word]) for word in query_tokens if word in indexed_vocabulary]\n\n# Calculate relevance scores for each document\nrelevance_scores = []\nfor idx, row in df.iterrows():\n    tf_idf_doc = r

In [ ]:
#takes too long to run


# Hardcoded user query
user_query = "information retrieval model"

# Tokenize the user query
query_tokens = user_query.split()

# Calculate TF/IDF weights for the query
tf_idf_query = [(indexed_vocabulary[word], query_tokens.count(word) * idf[word]) for word in query_tokens if word in indexed_vocabulary]

# Function to create a vector representation of a document/query in the vector space model
def create_vector(tf_idf_weights, vocabulary_size):
    vector = [0] * vocabulary_size  #initializing vector with zero
    for term_index, weight in tf_idf_weights:
        vector[term_index] = weight
    return vector

# Create vectors for the query and documents
query_vector = create_vector(tf_idf_query, len(indexed_vocabulary))
document_vectors = [create_vector(doc['TF_IDF_WEIGHTS'], len(indexed_vocabulary)) for _, doc in df.iterrows()]

# Calculate relevance scores
relevance_scores = []
for doc_id, document_vector in enumerate(document_vectors):
    # Calculate the inner product (scalar product) of the query vector with the document vector
    inner_product = sum(query_weight * document_weight for query_weight, document_weight in zip(query_vector, document_vector))
    relevance_scores.append((doc_id, inner_product))

# Sort relevance scores in descending order
relevance_scores = sorted(relevance_scores, key=lambda x: x[1], reverse=True)

# Print the top k relevant documents
top_k = 5
print(f"Top {top_k} Relevant Documents for Query: '{user_query}':")
for doc_id, score in relevance_scores[:top_k]:
    print(f"Document ID: {doc_id}, Relevance Score: {score}")


'# Hardcoded user query\nuser_query = "information retrieval model"\n\n# Tokenize the user query\nquery_tokens = user_query.split()\n\n# Calculate TF/IDF weights for the query\ntf_idf_query = [(indexed_vocabulary[word], query_tokens.count(word) * idf[word]) for word in query_tokens if word in indexed_vocabulary]\n\n# Function to create a vector representation of a document/query in the vector space model\ndef create_vector(tf_idf_weights, vocabulary_size):\n    vector = [0] * vocabulary_size  # Initialize vector with zeros\n    for term_index, weight in tf_idf_weights:\n        vector[term_index] = weight\n    return vector\n\n# Create vectors for the query and documents\nquery_vector = create_vector(tf_idf_query, len(indexed_vocabulary))\ndocument_vectors = [create_vector(doc[\'TF_IDF_WEIGHTS\'], len(indexed_vocabulary)) for _, doc in df.iterrows()]\n\n# Calculate relevance scores\nrelevance_scores = []\nfor doc_id, document_vector in enumerate(document_vectors):\n    # Calculate the 